#Title

In [1]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('input.csv')
df

,Location,Area,Team Member,Start Date,Start Time,End Date,End Time,Total Meal Break,Total Rest Break,Total Time,Status,Comment,Cost
0,BiFOR Birra Forlì,CUCINA,Enzo Pilogallo,2022-06-01,16:00,2022-06-02,00:00,0:00,0:00,8.0,Unpublished,NaN,0.0
1,BiFOR Birra Forlì,CUCINA,THOMAS VERSARI,2022-06-01,16:30,2022-06-02,01:00,0:00,0:00,8.5,Unpublished,NaN,127.5
2,BiFOR Birra Forlì,BAR,AMEDEO SANTOLINI,2022-06-01,17:30,2022-06-02,03:00,0:00,0:00,9.5,Unpublished,NaN,142.5
3,BiFOR Birra Forlì,SALA,DANIELE TASSI,2022-06-01,18:00,2022-06-02,01:00,0:00,0:00,7.0,Unpublished,NaN,105.0
4,BiFOR Birra Forlì,CUCINA,MARJORIE KOBEL DE TOLEDO,2022-06-01,18:30,2022-06-02,01:30,0:00,0:00,7.0,Unpublished,NaN,105.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,BiFOR Birra Forlì,CUCINA,MARJORIE KOBEL DE TOLEDO,2022-06-30,17:00,2022-07-01,01:30,0:00,0:00,8.5,Unpublished,Artusiana,127.5
270,BiFOR Birra Forlì,CUCINA,THOMAS VERSARI,2022-06-30,17:00,2022-07-01,01:00,0:00,0:00,8.0,Unpublished,NaN,120.0
271,BiFOR Birra Forlì,BAR,RICCARDO RICCIARDI,2022-06-30,17:30,2022-07-01,01:00,0:00,0:00,7.5,Unpublished,NaN,112.5
272,BiFOR Birra Forlì,BAR,ERIKA GNASSI,2022-06-30,19:30,2022-07-01,01:30,0:00,0:00,6.0,Unpublished,NaN,90.0


In [15]:
def conv_time_float(value):
    vals = value.split(':')
    t, hours = divmod(float(vals[0]), 24)
    t, minutes = divmod(float(vals[1]), 60)
    minutes = minutes / 60.0
    return hours + minutes

In [16]:
df['pausa'] = df['Total Meal Break'].apply(conv_time_float)
df['notturno'] = df['End Time'].apply(conv_time_float)
df['notturno'].mask(df['notturno'] > 7, 0, inplace=True)
df['totale'] = df['Total Time']
df['data'] = df['Start Date']

In [17]:
df = df[['Team Member','data','notturno','totale']]
df

,Team Member,data,notturno,totale
0,Enzo Pilogallo,2022-06-01,0.0,8.0
1,THOMAS VERSARI,2022-06-01,1.0,8.5
2,AMEDEO SANTOLINI,2022-06-01,3.0,9.5
3,DANIELE TASSI,2022-06-01,1.0,7.0
4,MARJORIE KOBEL DE TOLEDO,2022-06-01,1.5,7.0
...,...,...,...,...
269,MARJORIE KOBEL DE TOLEDO,2022-06-30,1.5,8.5
270,THOMAS VERSARI,2022-06-30,1.0,8.0
271,RICCARDO RICCIARDI,2022-06-30,1.0,7.5
272,ERIKA GNASSI,2022-06-30,1.5,6.0


In [18]:
from itertools import cycle

##repeat
df2 = df.loc[df.index.repeat(3)].reset_index(drop=True)

num_cycle = cycle([1, 2, 3])
df2['cat'] = [next(num_cycle) for num in range(len(df2))]

## repeat
df2 = df2.replace({'cat': {1: '_totale', 2: 'di cui notturni',3:'permessi'}})

df2['notturno'].mask(df2['cat'] == '_totale', 0, inplace=True)
df2['totale'].mask(df2['cat'] == 'di cui notturni', 0, inplace=True)
df2['notturno'].mask(df2['cat'] == 'permessi', 0, inplace=True)
df2['totale'].mask(df2['cat'] == 'permessi', 0, inplace=True)

df2

,Team Member,data,notturno,totale,cat
0,Enzo Pilogallo,2022-06-01,0.0,8.0,_totale
1,Enzo Pilogallo,2022-06-01,0.0,0.0,di cui notturni
2,Enzo Pilogallo,2022-06-01,0.0,0.0,permessi
3,THOMAS VERSARI,2022-06-01,0.0,8.5,_totale
4,THOMAS VERSARI,2022-06-01,1.0,0.0,di cui notturni
...,...,...,...,...,...
817,ERIKA GNASSI,2022-06-30,1.5,0.0,di cui notturni
818,ERIKA GNASSI,2022-06-30,0.0,0.0,permessi
819,DANIELE TASSI,2022-06-30,0.0,5.0,_totale
820,DANIELE TASSI,2022-06-30,1.5,0.0,di cui notturni


In [19]:
df2['ore'] = df2['notturno']+df2['totale']
df2

,Team Member,data,notturno,totale,cat,ore
0,Enzo Pilogallo,2022-06-01,0.0,8.0,_totale,8.0
1,Enzo Pilogallo,2022-06-01,0.0,0.0,di cui notturni,0.0
2,Enzo Pilogallo,2022-06-01,0.0,0.0,permessi,0.0
3,THOMAS VERSARI,2022-06-01,0.0,8.5,_totale,8.5
4,THOMAS VERSARI,2022-06-01,1.0,0.0,di cui notturni,1.0
...,...,...,...,...,...,...
817,ERIKA GNASSI,2022-06-30,1.5,0.0,di cui notturni,1.5
818,ERIKA GNASSI,2022-06-30,0.0,0.0,permessi,0.0
819,DANIELE TASSI,2022-06-30,0.0,5.0,_totale,5.0
820,DANIELE TASSI,2022-06-30,1.5,0.0,di cui notturni,1.5


In [20]:
df2 = df2[['Team Member','data','cat','ore']]
df2

,Team Member,data,cat,ore
0,Enzo Pilogallo,2022-06-01,_totale,8.0
1,Enzo Pilogallo,2022-06-01,di cui notturni,0.0
2,Enzo Pilogallo,2022-06-01,permessi,0.0
3,THOMAS VERSARI,2022-06-01,_totale,8.5
4,THOMAS VERSARI,2022-06-01,di cui notturni,1.0
...,...,...,...,...
817,ERIKA GNASSI,2022-06-30,di cui notturni,1.5
818,ERIKA GNASSI,2022-06-30,permessi,0.0
819,DANIELE TASSI,2022-06-30,_totale,5.0
820,DANIELE TASSI,2022-06-30,di cui notturni,1.5


In [21]:
# import datetime as dt
# df2[['data']].apply(lambda x: dt.datetime.strftime(x['data'], '%A'), axis=1)

In [22]:
multi = df2.set_index(['Team Member', 'cat'])
multi['data'] = multi['data'].astype(str)
multi = multi.pivot(columns='data')
multi = multi.replace(np.nan,0)
multi

ore                        \
data                               2022-06-01 2022-06-02 2022-06-03   
Team Member        cat                                                
AMEDEO SANTOLINI   _totale                9.5        0.0        8.0   
                   di cui notturni        3.0        0.0        1.0   
                   permessi               0.0        0.0        0.0   
CATERINA MILANDRI  _totale                6.0        0.0        0.0   
                   di cui notturni        2.5        0.0        0.0   
...                                       ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        0.5        2.0        3.0   
                   permessi               0.0        0.0        0.0   
THOMAS VERSARI     _totale                8.5        0.0        6.5   
                   di cui notturni        1.0        0.0        1.0   
                   permessi               0.0        0.0        0.0   

                                                                     \
data                               2022-06-04 2022-06-05 2022-06-06   
Team Member        cat                                                
AMEDEO SANTOLINI   _totale                8.5        0.0        8.5   
                   di cui notturni        3.0        0.0        1.5   
                   permessi               0.0        0.0        0.0   
CATERINA MILANDRI  _totale                0.0        7.0        0.0   
                   di cui notturni        0.0        1.5        0.0   
...                                       ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        0.0        1.5        0.0   
                   permessi               0.0        0.0        0.0   
THOMAS VERSARI     _totale                6.0        0.0        0.0   
                   di cui notturni        1.0        0.0        0.0   
                   permessi               0.0        0.0        0.0   

                                                                     \
data                               2022-06-07 2022-06-08 2022-06-09   
Team Member        cat                                                
AMEDEO SANTOLINI   _totale                0.0        6.5        6.5   
                   di cui notturni        0.0        1.5        1.5   
                   permessi               0.0        0.0        0.0   
CATERINA MILANDRI  _totale                0.0        0.0        5.0   
                   di cui notturni        0.0        0.0        0.0   
...                                       ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        2.0        0.0        0.0   
                   permessi               0.0        0.0        0.0   
THOMAS VERSARI     _totale                8.0        6.5        0.0   
                   di cui notturni        1.0        1.0        0.0   
                   permessi               0.0        0.0        0.0   

                                               ...                        \
data                               2022-06-10  ... 2022-06-21 2022-06-22   
Team Member        cat                         ...                         
AMEDEO SANTOLINI   _totale                7.5  ...        0.0        8.0   
                   di cui notturni        3.0  ...        0.0        3.0   
                   permessi               0.0  ...        0.0        0.0   
CATERINA MILANDRI  _totale                0.0  ...        0.0        5.5   
                   di cui notturni        0.0  ...        0.0        1.5   
...                                       ...  ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        0.0  ...        0.0        3.0   
                   permessi               0.0  ...        0.0        0.0   
THOMAS VERSARI     _totale                6.5  ...        0.0        0.0   
                   di cui notturni        1.0  ...        0.0        0.0   
                   permessi               0.0  ...        0.0        

In [24]:
multi.to_excel('ciao1.xlsx')